We start here: 
http://cgmartini.nl/index.php/tutorials-general-introduction-gmx5/bilayers-gmx5#Lipid-bilayer

and want to do something like this: 
http://cgmartini.nl/index.php/tutorials-general-introduction-gmx5/proteins-gmx5#membrane-protein

and download the gromacs-2016 tarball

# Bilayer self-assembly

```
cd ~/moldyn/MPflat/martini-tutorial/bilayer-lipidome-tutorial/spontaneous-assembly/initial_assembly
gmx insert-molecules -ci DPPC-em.gro -box 7.5 7.5 7.5 -nmol 128 -radius 0.21 -try 500 -o 128_noW.gro
[gedit/vi] dppc.top # (make sure the .itp files are here)
gmx grompp -f minimization.mdp -c 128_noW.gro -p dppc.top -o dppc-min-init.tpr
gmx mdrun -deffnm dppc-min-init -v -c 128_minimized.gro
gmx solvate -cp 128_minimized.gro -cs water.gro -o waterbox.gro -maxsol 768 -radius 0.21
[gedit/vi/other editor] dppc.top # (make sure it reflects the number of water added)
gmx grompp -f minimization.mdp -c waterbox.gro -p dppc.top -o dppc-min-solvent.tpr 
gmx mdrun -deffnm dppc-min-solvent -v -c minimized.gro
```
then launched actual self-assembly run 
```
#!/bin/bash
#
#SBATCH --job-name=MPflattest
#SBATCH --output=test_%j.log
#SBATCH --dependency=singleton
#
## Time
##SBATCH --qos=long
##SBATCH --time=168:00:0
## CPU cores
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
## Max RAM
##SBATCH --mem-per-cpu=8G
## GPUs (same socket)
#SBATCH --gres gpu:1
#SBATCH --gres-flags=enforce-binding

module load GCC CUDA/8.0.44 binutils/2.25
source /home/stanford/levittm/shared/software/gromacs/bin/GMXRC

# self-assembly run

srun gmx grompp -f martini_md.mdp -c minimized.gro -p dppc.top -o dppc-md.tpr
srun gmx mdrun -deffnm dppc-md -v
```